In [ ]:
pip install pyvis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd

In [ ]:
from pyvis.network import Network
net = Network('1000px', '1000px')

In [ ]:
plt.rcParams['figure.figsize'] = [7, 7]

In [ ]:
f = open('deezer_edges.json')
graphs = json.load(f)
len(graphs)

9629

In [ ]:
labels = np.genfromtxt('deezer_target.csv', delimiter=',')[1:]
labels.shape

(9629, 2)

In [ ]:
labels[3]

array([3., 0.])

In [ ]:
def count_vertex(edges):
  ans = 0
  for edge in edges:
    if ans < np.max(edges):
      ans = np.max(edges)
  return ans + 1
print(count_vertex(graphs['1']))

18


In [ ]:
def create_adjacent_list(edges, size):
  adj = [[] for i in range(size)]
  for edge in edges:
    adj[edge[0]].append(edge[1])
    adj[edge[1]].append(edge[0])
  return adj
create_adjacent_list(graphs['1'], count_vertex(graphs['1']))

[[6],
 [8, 16, 6, 10, 13, 15],
 [6],
 [6, 12, 14],
 [6],
 [6, 16, 15, 13],
 [0, 1, 2, 3, 4, 5, 14, 7, 8, 9, 10, 12, 13, 11, 15, 16, 17],
 [6],
 [1, 6, 10, 15, 17, 13],
 [6],
 [1, 6, 8, 16, 13, 15, 17],
 [6],
 [3, 6, 14],
 [1, 5, 6, 8, 10, 15, 16],
 [3, 6, 12],
 [1, 5, 6, 8, 10, 13, 16],
 [1, 5, 6, 10, 13, 15],
 [6, 8, 10]]

In [ ]:
"""
very simple implementation for comb function (only m = 2, 3, 4 used)
"""
def comb(n, m):
  def fact(n):
    if n == 0:
      return 1
    ans = 1
    for i in range(1, n + 1):
      ans *= i
    return ans
  
  if m == 0:
    return 1
  if n < m:
    return 0
  return int(fact(n) / (fact(n - m) * fact(m)))

In [ ]:
def count_cliques(X, tri, adjacent):
  ans = 0
  for v in tri:
    for u in adjacent[v]:
      if X[u] == 2:
        ans += 1
    X[v] = 0
  return ans
def count_cycles(X, star0, adjacent):
  ans = 0
  for v in star0:
    for u in adjacent[v]:
      if X[u] == 3:
        ans += 1
    X[v] = 0
  return ans

In [ ]:
def count_graphlet(edges, adjacent, size):
  T, S, N, cliques, cycles = 0, 0, 0, 0, 0
  T_T, S0_S1, T_S0orS1, S_S = 0, 0, 0, 0
  T_I, S0orS1_I, I_I, I_I_1 = 0, 0, 0, 0
  for edge in edges:
    X = np.zeros(size)
    tri, star0, star1= [], [], []
    for v in adjacent[edge[0]]:
      if (v == edge[1]):
        continue
      X[v] = 1
      star0.append(v)
    for v in adjacent[edge[1]]:
      if (v == edge[0]):
        continue
      if X[v] == 1:
        X[v] = 2
        star0.remove(v)
        tri.append(v)
      else:
        X[v] = 3
        star1.append(v)
    cliques += count_cliques(X.copy(), tri, adjacent)
    cycles += count_cycles(X.copy(), star0, adjacent)

    tri_l, star0_l, star1_l = len(tri), len(star0), len(star1)

    T_T += comb(tri_l, 2)
    S0_S1 += star0_l * star1_l
    T_S0orS1 += tri_l * (star0_l + star1_l)
    S_S += comb(star0_l, 2) + comb(star1_l, 2)
    T_I += tri_l * (size - tri_l - star0_l - star1_l - 2)
    S0orS1_I += (star0_l + star1_l) * (size - tri_l - star0_l - star1_l - 2)
    I_I += comb(size - tri_l - star0_l - star1_l - 2, 2)
    I_I_1 += len(edges) - ((star0_l + tri_l) + (star1_l + tri_l) + 1)
    T += tri_l
    S += star0_l + star1_l
    N += size - tri_l - star0_l - star1_l - 2
  return  T_T, S0_S1, T_S0orS1, S_S, T_I, S0orS1_I, I_I, I_I_1, T, S, N, cliques, cycles

     
graph = [[0, 1], [1, 2], [2, 3], [0, 3]]
T_T, S0_S1, T_S0orS1, S_S, T_I, S0orS1_I, I_I, I_I_1, T, S, N, cliques, cycles = count_graphlet(graph, create_adjacent_list(graph, 4), 4)
T_T, S0_S1, T_S0orS1, S_S, T_I, S0orS1_I, I_I, I_I_1, T, S, N, cliques, cycles

(0, 4, 0, 0, 0, 0, 0, 4, 0, 8, 0, 0, 4)

In [ ]:
def extract_features(cliques, cycles, T_T, S0_S1, T_S0orS1, S_S,
T_I, S0orS1_I, I_I, I_I_1, T, S, N, label, size):
  G3_1 = T / 3
  G3_2 = S / 2
  G3_3 = N
  G3_4 = comb(size, 3) - G3_1 - G3_2 - G3_3

  G4_1 = cliques / 6
  G4_2 = T_T - cliques
  G4_3 = (T_S0orS1 - 4 * G4_2)/ 2
  G4_4 = cycles / 4
  G4_5 = (S_S - G4_3) / 3
  G4_6 = S0_S1 - (4 * G4_4)
  G4_7 = (T_I - G4_3) / 3
  G4_8 = (S0orS1_I - (2 * G4_6)) / 2
  G4_9 = (I_I_1 - (6 * G4_1 + 4 * G4_2 + 2 * G4_3 + 4 * G4_4 + 2 * G4_6)) / 2
  G4_10 = I_I - (2 * G4_9)
  G4_11 = comb(size, 4) - (G4_1 + G4_2 + G4_3 + G4_4 +
                                  G4_5 + G4_6 + G4_7 + G4_8 +
                                  G4_9 + G4_10)
  ans = [G3_1, G3_2, G3_3, G3_4, G4_1, G4_2, G4_3, G4_4,
         G4_5, G4_6, G4_7, G4_8, G4_9, G4_10, G4_11, label]
  return np.array(ans)
extract_features(cliques, cycles, T_T, S0_S1, T_S0orS1, S_S,
                   T_I, S0orS1_I, I_I, I_I_1, T, S, N, 1, 4)

array([0., 4., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.])

In [ ]:
n = 7
graph = [[0, 1], [0, 2], [0, 3], [0, 6], [1, 2], [1, 4], [1, 5], [2, 6], [3, 4], [3, 5], [4, 6], [5, 6]]
for i in range(n):
  net.add_node(i, label=f'Node{i}', size=25)
for edge in graph:
  net.add_edge(edge[0], edge[1])
net.show('example.html')

In [ ]:
def find_features(graph, label, n):
  adjacent_list = create_adjacent_list(graph, n)
  T_T, S0_S1, T_S0orS1, S_S, T_I, S0orS1_I, I_I, I_I_1, T, S, N, cliques, cycles = count_graphlet(graph, adjacent_list, n)
  ans = extract_features(cliques, cycles, T_T, S0_S1, T_S0orS1, S_S,
                   T_I, S0orS1_I, I_I, I_I_1, T, S, N, label, n)
  return ans

In [ ]:
find_features(graph, 1, n)

array([ 2., 24.,  6.,  3.,  0.,  1.,  6., 11.,  8.,  6.,  0.,  2.,  0.,
        1.,  0.,  1.])

In [ ]:
def make_random_graph(edge_size, size):
  all_possible = [(i, j) for i in range(size) for j in range(i + 1, size)]
  indexes =  np.random.choice([i for i in range(comb(size, 2))], size=edge_size, replace=False)
  return (np.array(all_possible))[indexes]
make_random_graph(10, 5)

array([[0, 2],
       [2, 4],
       [2, 3],
       [1, 4],
       [0, 1],
       [1, 2],
       [0, 3],
       [3, 4],
       [0, 4],
       [1, 3]])

In [ ]:
def calculate_SRP(edge_size, main_features, size, number_of_random_graphs, epsilon=4):
  label = main_features[-1]
  features = np.zeros(shape=(15,))
  for i in range(number_of_random_graphs):
    graph = make_random_graph(edge_size, size)
    features += find_features(graph, 0, n)[:-1]
  features = features / number_of_random_graphs
  delta = (main_features[:-1] - features) / (main_features[:-1] + features + epsilon)
  SRP = delta / np.linalg.norm(delta)
  return np.concatenate([SRP, [label]])


In [ ]:
data = np.zeros(shape=(16,))
NUMBER_OF_RANDOM_GRAPHS = 10
for index in graphs:
  graph = graphs[index]
  n = count_vertex(graph)
  features = find_features(graph, labels[int(index)][1], n)
  features = calculate_SRP(len(graph), features, n, NUMBER_OF_RANDOM_GRAPHS, epsilon=4)
  data = np.vstack((data, features))
  

In [ ]:
data = data[1:]

In [ ]:
data[1]

array([ 0.24353296,  0.03087628, -0.10103096,  0.04922456,  0.41271341,
        0.20726632,  0.24442925, -0.39103598,  0.32120253, -0.43432568,
        0.19640484, -0.29550759, -0.25329654, -0.02358811,  0.13688159,
        1.        ])

In [ ]:
columns = ['G3_1', 'G3_2', 'G3_3', 'G3_4', 'G4_1', 'G4_2', 'G4_3', 'G4_4', 'G4_5',
           'G4_6', 'G4_7', 'G4_8', 'G4_9', 'G4_10', 'G4_11', 'Label']
df = pd.DataFrame(data, columns=columns)

In [ ]:
df.head() 

,G3_1,G3_2,G3_3,G3_4,G4_1,G4_2,G4_3,G4_4,G4_5,G4_6,G4_7,G4_8,G4_9,G4_10,G4_11,Label
0,0.359460,0.227639,-0.058806,0.003365,0.400293,0.398230,0.387503,0.211338,0.387343,0.009117,0.333568,0.039954,-0.183240,-0.072063,0.010188,0.0
1,0.243533,0.030876,-0.101031,0.049225,0.412713,0.207266,0.244429,-0.391036,0.321203,-0.434326,0.196405,-0.295508,-0.253297,-0.023588,0.136882,1.0
2,0.274950,0.050286,-0.102843,0.039747,0.468500,0.364639,0.249897,-0.335416,0.301619,-0.318409,0.167981,-0.125492,-0.355005,-0.079102,0.117049,0.0
3,0.167169,-0.091235,-0.062852,0.137834,0.409308,0.112831,-0.028614,-0.412638,0.101802,-0.453190,0.185869,-0.053293,-0.438431,0.121464,0.359613,0.0
4,0.047744,0.216933,-0.286313,0.045998,-0.010480,0.128350,0.251009,-0.128154,0.396545,-0.389061,-0.342887,-0.317630,-0.398780,-0.262929,0.123790,1.0


In [ ]:
df.to_csv('deezer_ego_dataset.csv', sep=',')

In [ ]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
y = df['Label']
X_not_normalized = df.drop(columns=['Label'], axis=1)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
# y.head()
X_not_normalized.head()

,G3_1,G3_2,G3_3,G3_4,G4_1,G4_2,G4_3,G4_4,G4_5,G4_6,G4_7,G4_8,G4_9,G4_10,G4_11
0,0.359460,0.227639,-0.058806,0.003365,0.400293,0.398230,0.387503,0.211338,0.387343,0.009117,0.333568,0.039954,-0.183240,-0.072063,0.010188
1,0.243533,0.030876,-0.101031,0.049225,0.412713,0.207266,0.244429,-0.391036,0.321203,-0.434326,0.196405,-0.295508,-0.253297,-0.023588,0.136882
2,0.274950,0.050286,-0.102843,0.039747,0.468500,0.364639,0.249897,-0.335416,0.301619,-0.318409,0.167981,-0.125492,-0.355005,-0.079102,0.117049
3,0.167169,-0.091235,-0.062852,0.137834,0.409308,0.112831,-0.028614,-0.412638,0.101802,-0.453190,0.185869,-0.053293,-0.438431,0.121464,0.359613
4,0.047744,0.216933,-0.286313,0.045998,-0.010480,0.128350,0.251009,-0.128154,0.396545,-0.389061,-0.342887,-0.317630,-0.398780,-0.262929,0.123790


In [ ]:
X = (X_not_normalized - X_not_normalized.min()) / (X_not_normalized.max()-  X_not_normalized.min())
X

,G3_1,G3_2,G3_3,G3_4,G4_1,G4_2,G4_3,G4_4,G4_5,G4_6,G4_7,G4_8,G4_9,G4_10,G4_11
0,0.992988,0.809675,0.566846,0.272717,0.718287,0.898832,0.964193,0.887897,0.819602,0.803743,0.888929,0.559995,0.334124,0.314340,0.402668
1,0.747890,0.424074,0.498402,0.376553,0.731614,0.674005,0.798341,0.342496,0.760529,0.245866,0.770512,0.235947,0.278607,0.357765,0.527047
2,0.814314,0.462112,0.495465,0.355093,0.791471,0.859284,0.804680,0.392856,0.743039,0.391695,0.745973,0.400178,0.198007,0.308035,0.507576
3,0.586438,0.184770,0.560288,0.577184,0.727960,0.562824,0.481829,0.322937,0.564573,0.222133,0.761416,0.469921,0.131895,0.487704,0.745709
4,0.333944,0.788693,0.198071,0.369247,0.277543,0.581095,0.805968,0.580514,0.827821,0.302812,0.304925,0.214577,0.163317,0.143361,0.514194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9624,0.612558,0.837337,0.307010,0.326123,0.433659,0.635916,0.919724,0.497883,0.840489,0.262022,0.528400,0.178333,0.203133,0.210126,0.463543
9625,0.759214,0.618920,0.427998,0.342666,0.607105,0.696800,0.902086,0.414952,0.802531,0.254842,0.735225,0.184331,0.267512,0.300622,0.482137
9626,0.437287,0.634157,0.357558,0.407965,0.288788,0.440326,0.841654,0.545438,0.840709,0.205974,0.264688,0.097622,0.277307,0.324236,0.550945
9627,0.285616,0.737963,0.205763,0.419357,0.288788,0.368133,0.792486,0.632788,0.837366,0.304133,0.329959,0.089058,0.196404,0.202413,0.569439


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
def learn(n_estimators=100):
  model = RandomForestClassifier(n_estimators, random_state=1, max_depth=2)
  # model = DecisionTreeClassifier(max_depth=4, random_state=1)
  # model = GaussianNB()
  # model = KNeighborsClassifier(n_neighbors=51)
  # model = RandomForestClassifier(max_depth=4, random_state=1)
  # model = SVC(gamma='auto')
  # model = BaggingClassifier()
  model.fit(X_train, y_train)
  prediction = model.predict(X_test)
  return accuracy_score(prediction, y_test)
learn()

0.5799584631360333

In [ ]:
def fine_tune():
  n_estimators = np.arange(2, 101, 1)
  max, opt = 0, 0
  for x in n_estimators:
    ans = learn(x)
    if max < ans:
      opt = x
      max = ans
  return opt, max
fine_tune()

(4, 0.5815160955347871)